In [1]:
from pyscf import gto, scf
#import os
from ad_afqmc import pyscf_interface, driver, mpi_jax,  run_afqmc
from ad_afqmc import config
atom = '''
 O   -1.485163346097   -0.114724564047    0.000000000000
 H   -1.868415346097    0.762298435953    0.000000000000
 H   -0.533833346097    0.040507435953    0.000000000000
'''

mol = gto.Mole(
    verbose=4,
    atom=atom,
    basis='ccpvdz',
    )

mol.build()
mf = scf.RHF(mol).density_fit()
mf.kernel()

# moi,_,stable,_ = mf.stability(return_status=True)
# if not stable: mf = mf.newton().run(moi, mf.mo_occ)
#else: print('HF RESULT IS STABLE')
    



# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#1882-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Mon Jun 30 11:52:56 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O     -1

-76.02658211086111

In [25]:
from pyscf import mp
mmp = mp.MP2(mf)
e_mp2 = mmp.kernel()[0]


******** <class 'pyscf.mp.dfmp2.DFMP2'> ********
nocc = 5, nmo = 24
max_memory 4000 MB (current use 51 MB)
E(DFMP2) = -76.2307451323285  E_corr = -0.204163021467412
E(SCS-DFMP2) = -76.2268416334085  E_corr = -0.200259522547366
E_corr(same-spin) = -0.0516185806309946
E_corr(oppo-spin) = -0.152544440836418


In [27]:
mmp.e_corr

-0.20416302146741214

In [26]:
print(e_mp2)

-0.20416302146741214


In [4]:
# ad afqmc

pyscf_interface.prep_afqmc(mf)
options = {
    "dt":0.001,
    "n_eql": 1,
    "n_ene_blocks": 1,
    "n_sr_blocks": 50,
    "n_blocks": 1,
    "n_walkers": 50,
    "seed": 98,
    "trial": "rhf",
    "walker_type": "rhf",
    #"LNO":True,
    # "ad_mode": "reverse",
}
#pyscf_interface.prep_afqmc(mf)
#from mpi4py import MPI

#MPI.Finalize()
e_afqmc, err_afqmc = run_afqmc.run_afqmc_lno_mf(options=options)

#
# Preparing AFQMC calculation
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (5, 5)
# Number of basis functions: 7
# Number of Cholesky vectors: 104
#
# Hostname: YICHI
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 7
# nelec: (5, 5)
#
# dt: 0.001
# n_eql: 2
# n_ene_blocks: 1
# n_sr_blocks: 50
# n_blocks: 3
# n_walkers: 50
# seed: 98
# trial: rhf
# walker_type: rhf
# n_prop_steps: 50
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# free_projection: False
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#
# Equilibration sweeps:
#   Iter        Block energy      Walltime
#     0      -7.567857497e+01     1.57e+00 
#     1      -7.572489929e+01     6.00e+00 
#     2      -7.572996521e+01     1.01e+01 
#
# Sampling sweeps:
#  Iter        Mean energy          Stochastic error       Walltime
     0      -

In [15]:
from ad_afqmc.lno.afqmc import LNOAFQMC
filename = 'fragmentenergies.txt'
for thresh in [1e-5]:
    f = open(filename,'w')
    f.close()
    mfcc = LNOAFQMC(mf, thresh=thresh, frozen=1).set(verbose=5)
    mfcc.thresh_occ = thresh*10
    mfcc.thresh_vir = thresh
    mfcc.nblocks = 1
    mfcc.nwalk_per_proc = 5#10
#    mfcc.seed = 1234 
#    mfcc.runfrags=[4]
    mfcc.lo_type = 'pm'
    mfcc.no_type = 'cim'
    mfcc.frag_lolist = '1o'
    mfcc.force_outcore_ao2mo = True
#    mfcc.vmc_root = vmc_root
    mfcc.kernel() #canonicalize=False,chol_vecs=chol_vecs)
    #ecc = mfcc.e_corr #+ mf.energy_nuc()
    # ecc_pt2corrected = mfcc.e_corr_pt2corrected(mmp.e_corr)
    # ecc_t = mfcc.e_corr_ccsd_t
    # log.info('thresh = %.0e  E_corr(AFQMC)     = %.10f, %.10f  rel(wrt CCSD) = %6.2f%%  '
    #         'diff = % .10f',
    #         thresh, ecc,mf.energy_nuc(), ecc/mcc.e_corr*100, ecc-mcc.e_corr)

    # log.info('thresh = %.0e  E_corr(AFQMC) = %.10f  E_corr(AFQMC+PT2) = %.10f', thresh, ecc, ecc_pt2corrected)


******** <class 'ad_afqmc.lno.afqmc.afqmc.LNOAFQMC'> ********
nocc = 4, nmo = 23
frozen orbitals 1
max_memory 4000 MB (current use 219 MB)
thresh_occ = 1.000000e-04
thresh_vir = 1.000000e-05
force_outcore_ao2mo = True
no_type = ie
Constructing LOs
lo_type = pm
LOs span exactly the occupied space.
Using single-LO fragment
nfrag = 4  nlo = 4
frag_atmlist = None
frag_lolist = [[0], [1], [2], [3]]
frag_wghtlist = [1. 1. 1. 1.]
frag_nonvlist = None
    CPU time for LO and fragment        0.33 sec, wall time      0.29 sec
ao2mo est mem= 0.13 MB  avail mem= 3779.28 MB
Lov is saved to /tmp/zj0p1rz9
    CPU time for transforming DF-MP2 integrals      0.00 sec, wall time      0.00 sec
    CPU time for Integral xform         0.02 sec, wall time      0.00 sec
    CPU time for make pno               0.00 sec, wall time      0.00 sec
Solving impurity problem
#
# Preparing LNO-AFQMC calculation
# Calculating Cholesky integrals
# Generating Cholesky Integrals
# frozen orbitals are [ 0 21 22 23]
# loc

In [20]:
mfcc.e_corr_pt2corrected(mmp.e_corr)-(-0.2294306829571724)

-0.002801314305351882

In [14]:
from ad_afqmc.lno.cc import LNOCCSD
for thresh in [1e-5]:
    lnocc = LNOCCSD(mf, thresh=thresh)
    lnocc.thresh_occ = thresh*10
    lnocc.thresh_vir = thresh
    lnocc.lo_type = 'pm'
    lnocc.no_type = 'cim'
    lnocc.frag_lolist = '1o'
    lnocc.frag_atmlist = None 
    lnocc.ccsd_t = True
    lnocc.force_outcore_ao2mo = True
    lnocc.kernel()
    ecc = lnocc.e_corr
    ecc_pt2corrected = lnocc.e_corr_pt2corrected(mmp.e_corr)


******** <class 'ad_afqmc.lno.cc.ccsd.LNOCCSD'> ********
nocc = 5, nmo = 24
max_memory 4000 MB (current use 218 MB)
thresh_occ = 1.000000e-04
thresh_vir = 1.000000e-05
force_outcore_ao2mo = True
no_type = ie
Constructing LOs
lo_type = pm
LOs span exactly the occupied space.
Using single-LO fragment
nfrag = 5  nlo = 5
frag_atmlist = None
frag_lolist = [[0], [1], [2], [3], [4]]
frag_wghtlist = [1. 1. 1. 1. 1.]
frag_nonvlist = None
Lov is saved to /tmp/y6u87n1p
Fragment 1  E_corr(MP2) = -0.000288810960515213  E_corr(CCSD) = -0.000283662265533924  E_corr(CCSD(T)) = -1.41768556195383e-23
Fragment 2  E_corr(MP2) = -0.0512673231004163  E_corr(CCSD) = -0.0543312579164522  E_corr(CCSD(T)) = -0.000885958031755243
Fragment 3  E_corr(MP2) = -0.0515265352099154  E_corr(CCSD) = -0.054631551262891  E_corr(CCSD(T)) = -0.000900246687300221
Fragment 4  E_corr(MP2) = -0.0450327030738217  E_corr(CCSD) = -0.0471601599260646  E_corr(CCSD(T)) = -0.000702283214904525
Fragment 5  E_corr(MP2) = -0.053585408451

In [21]:
mfcc.e_corr_pt2corrected(mmp.e_corr)-(-0.211397710372792)

-0.020834286889732268